In [1]:
import pandas as pd
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [2]:
df = pd.read_csv('data/Dataset for participants V2/test.csv')
testdf = pd.DataFrame(df['tag_id'])
# testdf['imagefn'] = df.apply(lambda row: '{}.bmp'.format(row['tag_id']),axis=1)
# df2.to_csv('data/Dataset for participants V2/v2.result.csv',index=False)
testdf.head(3)

,tag_id
0,18969
1,23148
2,38223


In [3]:
model = load_model('models/mafat_model.final.h5')
image_size=224

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator=test_datagen.flow_from_dataframe(
    dataframe=testdf,
    directory='data/test',
    x_col='tag_id',
    y_col=None,
    has_ext=False,
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(image_size, image_size))

test_generator.reset()
pred=model.predict_generator(test_generator,
                             steps=5,#test_generator.samples,
                             verbose=1)

labels = ['small vehicle','large vehicle','minibus','hatchback','sedan','bus',
           'minivan','truck','van','jeep','cement mixer','dedicated agricultural vehicle','tanker',
           'crane truck','pickup','light truck','prime mover','red','black','blue','silver/grey','white',
           'other','yellow','green','sunroof','luggage_carrier','open_cargo_area','enclosed_cab','spare_wheel',
           'wrecked','flatbed','ladder','enclosed_box','soft_shell_box','harnessed_to_a_cart','ac_vents']
results = pd.DataFrame(data=pred,columns=labels)

Found 11879 images.
5/5 [==============================] - 3s 516ms/step


In [33]:
df_answer = pd.read_csv('answer.csv.empty')
results['tag_id'] = testdf['tag_id']
for label in labels:
    sorted_by_feature = results.sort_values(by=label, ascending=False)
    tags = sorted_by_feature['tag_id'].tolist()
    df_answer[label] = tags

df_answer.to_csv('answer.csv',index=False)
df_answer.head(5)

,small vehicle,large vehicle,minibus,hatchback,sedan,bus,minivan,truck,van,jeep,...,open_cargo_area,enclosed_cab,spare_wheel,wrecked,flatbed,ladder,enclosed_box,soft_shell_box,harnessed_to_a_cart,ac_vents
0,18978,23148,18969,14938,18978,23148,38223,23148,18969,14938,...,38223,23148,23148,23148,23148,18969,23148,23148,23148,23148
1,14938,18969,23148,23148,18969,14938,14938,18969,23148,23148,...,18969,18969,38223,18969,14938,23148,18969,18969,18969,18969
2,38223,14938,38223,18978,38223,18969,23148,14938,38223,38223,...,23148,38223,18969,14938,18969,38223,18978,14938,14938,38223
3,18969,38223,14938,38223,23148,18978,18969,38223,14938,18978,...,14938,14938,14938,38223,38223,14938,38223,38223,38223,18978
4,23148,18978,18978,18969,14938,38223,18978,18978,18978,18969,...,18978,18978,18978,18978,18978,18978,14938,18978,18978,14938
